# Checkpointing in FEniCSx

## Jørgen Schartum Dokken

### Simula Research Laboratory

FEniCS 23'

<div>
<p style="text-align:center;"><img src="figures/fenics_banner.png" width="500">
</div>


# What is checkpointing?

> Checkpointing refers to the ability to store the state of a computation in a way that allows it be continued at a later time without changing the computation’s behavior [1]

<div>
<p style="text-align:center;"><img src="figures/checkpointing.drawio.png" width="600">
</div>

[1] Schulz, M. (2011). Checkpointing. In: Padua, D. (eds) Encyclopedia of Parallel Computing. Springer, Boston, MA. [10.1007/978-0-387-09766-4_62](https://doi.org/10.1007/978-0-387-09766-4_62)

# What is checkpointing in FEniCSx?
- Write a mesh and function to file, that is readable by DOLFINx
- Implemented in Python with [ADIOS2](https://adios2.readthedocs.io/en/latest/)
- Available at: https://github.com/jorgensd/adios4dolfinx/

In [7]:
from dolfinx import mesh, fem
from mpi4py import MPI
import numpy as np
import adios4dolfinx as adx
from pathlib import Path

domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
V = fem.FunctionSpace(domain, ("Lagrange", 5))
u = fem.Function(V)
u.interpolate(lambda x: x[0]**5 + 3*x[1]*x[0]**2)

engine = "BP4"
checkpoint_file = Path("function_checkpoint.bp")
adx.write_mesh(domain, checkpoint_file, engine=engine)
adx.write_function(u, checkpoint_file, engine=engine)

print(f"File {str(checkpoint_file)} exists: {checkpoint_file.exists()}")
domain_in = adx.read_mesh(MPI.COMM_WORLD, checkpoint_file, engine, mesh.GhostMode.shared_facet)
V_in = fem.FunctionSpace(domain_in, ("Lagrange", 5))
u_in = fem.Function(V_in)
adx.read_function(u_in, checkpoint_file, engine=engine)
print(f"Max difference: {max(abs(u_in.x.array - u.x.array))}")

File function_checkpoint.bp exists: True
Max difference: 0.0


# Why do we need a new mesh when reading in a checkpoint?

# What is the optimal format?

## FEM software
- Most finite element software use their own format for inputting/outputting meshes and functions
- No general consensus on what format to use

## Post-processing
- Most formats are based on [VTK](https://vtk.org/) in some shape or form
- Many different file formats support `vtk`
  - `.pvd`-files (`xml`-based)
  - `.xdmf`-files (`xml`+binary (`.h5`)
  - `.bp`-files (binary files in folder)